The block below will generate a crossover network

In [ ]:
%run "fir_crossover_taps_generator.py" -fc 270 -Bt 460 -Aa 80 -fs 48000

The block below will plot a generated crossover network

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
pd.options.plotting.backend = 'plotly'

#Set sampling frequency
fs = 48000

#Read data
h_lp = np.genfromtxt('../data/lowpass_crossover.txt')
h_lp_fxp = np.genfromtxt('../data/lowpass_crossover_fxp.txt')*2**-15
h_hp = np.genfromtxt('../data/highpass_crossover.txt')
h_hp_fxp = np.genfromtxt('../data/highpass_crossover_fxp.txt')*2**-15

# Graph controls
dB = True
logX = False

#Drawing the filter
f, H_lp = signal.freqz(h_lp, 1, worN = 2**14, fs = fs)
f, H_lp_fxp = signal.freqz(h_lp_fxp, 1, worN = 2**14, fs = fs)
f, H_hp = signal.freqz(h_hp, 1, worN = 2**14, fs = fs)
f, H_hp_fxp = signal.freqz(h_hp_fxp, 1, worN = 2**14, fs = fs)

#Filter dataframe
df = pd.DataFrame(
    {
        'Low pass': abs(H_lp) if not dB else 20*np.log10(abs(H_lp)), 
        'Low pass fxp': abs(H_lp_fxp) if not dB else 20*np.log10(abs(H_lp_fxp)), 
        'High pass': abs(H_hp) if not dB else 20*np.log10(abs(H_hp)),
        'High pass fxp': abs(H_hp_fxp) if not dB else 20*np.log10(abs(H_hp_fxp)),
        'Sum' : abs(H_hp+H_lp) if not dB else 20*np.log10(abs(H_hp+H_lp)),
        'Sum fxp' : abs(H_hp_fxp+H_lp_fxp) if not dB else 20*np.log10(abs(H_hp_fxp+H_lp_fxp)),
    }
).set_index(f)

#Plotting 
fig = df.plot(
    labels = dict(index='Frequency', value='Magnitude', variable='Filter'), 
    template='plotly_dark',
    log_x=logX
)

#Formating
fig.update_layout(
    title = 'Crossover filters',
    xaxis = dict(ticksuffix='Hz', range=[20, 20000] if not logX else [np.log10(20), np.log10(20000)]),
    yaxis = dict(ticksuffix='dB') if dB else dict(ticksuffix=''),
)
fig